## Assegnare una variabile in maniera dinamica

In [ ]:
par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)

In [ ]:
for file in parametriche_files:
    file_name = file.split(".")[0]
    globals()[file_name] = pd.read_excel(parametriche + "\\" + file)

## Creare una nuova colonna con il metodo assign  
Il metodo .assign(**{"Codice PRODOTTO": lambda x: x["Codice PRODOTTO"].astype(str)}) viene utilizzato per assegnare una nuova colonna chiamata "Codice PRODOTTO" al DataFrame risultante. La colonna "Codice PRODOTTO" viene ottenuta dalla colonna originale "Codice PRODOTTO" del DataFrame selezionato, ma viene convertita in formato di stringa (str) utilizzando la funzione astype(str).  
  
I due asterischi ** sono un operatore chiamato "unpacking" o "keyword argument unpacking" in Python. Nel contesto del codice fornito, viene utilizzato per passare gli argomenti come parole chiave al metodo .assign().

Quando si utilizza ** prima di un oggetto di tipo dizionario ({}), viene effettuato l'unpacking del dizionario in una serie di argomenti chiave-valore separati. In pratica, consente di passare un numero variabile di argomenti come parole chiave a una funzione o a un metodo.

Nel caso specifico del codice, **{"Codice PRODOTTO": lambda x: x["Codice PRODOTTO"].astype(str)} viene utilizzato per specificare che la colonna "Codice PRODOTTO" del DataFrame risultante deve essere assegnata utilizzando la funzione lambda lambda x: x["Codice PRODOTTO"].astype(str), che converte i valori della colonna in stringhe.

In [ ]:
cod_prod = pd.DataFrame(par["Classificazione_codice_prodotto"]).loc[:,['MDM', 'GTM BDG', 'Solution','Sub solution',"Codice PRODOTTO"]].assign(**{"Codice PRODOTTO": lambda x: x["Codice PRODOTTO"].astype(str)})

cod_prod.rename(columns={"Codice PRODOTTO":"Codice Prodotto"}, inplace=True)

## Convertire le date espresse in formato numerico (da file xlsb)  
Se la condizione è vera, esegue le seguenti operazioni:
* Applica una funzione lambda alla colonna utilizzando il metodo .apply(). La funzione lambda converte i valori della colonna da una rappresentazione numerica di data (basata su giorni dal 1900) in una rappresentazione di data e ora utilizzando la formula (float(x) - 25569) * 86400.
* Riassegna i valori convertiti alla stessa colonna nel DataFrame cli.
* Utilizza il metodo pd.to_datetime() per convertire i valori della colonna in oggetti di tipo datetime, specificando l'unità di tempo come secondi utilizzando l'argomento unit="s".

In [ ]:
for col in cli.columns:
    if "Data" in col:
        
        cli[col] = cli[col].apply(lambda x: (float(x) - 25569) * 86400)
        cli[col] = pd.to_datetime(cli[col], unit="s")
    else:
        pass

## Concatenare i valori di una colonna riferiti ad un gruppo  

1) La colonna "Solution" viene convertita in una categoria ordinata utilizzando pd.Categorical(). Questo ordina le categorie in base all'ordine in cui compaiono nel DataFrame originale.
2) Il DataFrame viene ordinato in base alla colonna "Solution" utilizzando sort_values().
3) Viene creata una nuova colonna chiamata "Prodotti" che contiene la concatenazione delle categorie "Solution" per ogni "Cliente Merce". Questo viene fatto utilizzando groupby() per raggruppare il DataFrame per "Cliente Merce" e quindi applicando transform() per ogni gruppo. La funzione lambda all'interno di transform() unisce le categorie "Solution" di ciascun gruppo separandole con " + ".
4) Infine, viene applicata la funzione remov_duplicates() alla colonna "Prodotti" utilizzando apply(). La funzione remov_duplicates() sembra rimuovere eventuali duplicati all'interno della stringa di prodotti, mantenendo solo un'occorrenza di ciascun prodotto.

In [ ]:
def concat_prodotti(df_):
    df = clienti.loc[:,["Solution", "Cliente Merce"]].fillna("Senza attribuz.")
    df["Solution"] = pd.Categorical(df["Solution"], np.sort(df["Solution"].unique()))
    df.sort_values(by=["Solution"], inplace=True)
    df["Prodotti"] = df.groupby("Cliente Merce")["Solution"].transform(lambda x: " + ".join(x)).apply(lambda x: remov_duplicates(x))
    return df["Prodotti"]

## Verificare il formato di delle colonne di un Dataframe per applicarlo a quelle di un altro Dataframe con le stesse colonne

In [ ]:
for colonna in inflow.columns:
    formato = inflow[colonna][0]  # Assume che tutti i valori abbiano lo stesso formato
    if isinstance(formato, str) and isinstance(ins_man[colonna][0], str):
        ins_man[colonna] = ins_man[colonna].apply(lambda x: str(x).zfill(len(formato)))